In [ ]:
import mtcnn
# print version
print(mtcnn.__version__)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from datetime import datetime


In [ ]:
# Use the mtcnn library to create a face detector and extract faces for use with the FaceNet face detector model in subsequent sections
# Define the function extract_face() to load the photo from the loaded filename and return the extracted face. 
# It assumes the photo contains a face and will return the first face detected
from PIL import Image
from mtcnn.mtcnn import MTCNN
 
# Extract a face from a given image

def extract_face(filename,  required_size=(160,  160)):
    # Load the image as a NumPy array
    image = Image.open(filename)
    # Convert to RGB format
    image = image.convert('RGB')
    pixels = np.asarray(image)
    # Define detector with default weights
    detector = MTCNN()
    # Detect face
    results = detector.detect_faces(pixels)
    # Get the first face bounding box
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # Use coordinates to extract faces
    face  =  pixels[y1:y2, x1:x2]
    # Resize the face to the input size required by FaceNet 
    image  =  Image.fromarray(face)
    image  =  image.resize(required_size)
    face_array  =  np.asarray(image)
    # Result
    return  face_array

In [ ]:

import tensorflow as tf
from os import listdir
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
# We detect the face in each photo for the first 56 photos of Deepika_Padukone in the test dataset.
import numpy as np
folder = './data_bw/val/Deepika_Padukone/'
i = 1
# iterate over files
for filename in listdir(folder):
    # path
    if i<=56:
        path = folder + filename
        # extract face
        face = extract_face(path)
        print(i, face.shape)
        # plot
        plt.subplot(8, 7, i)
        plt.axis('off')
        plt.imshow(face)
        i += 1
plt.show()

In [ ]:
folder = './data_bw/val/Govinda/'
i = 1
# iterate over files
for filename in listdir(folder):
    # path
    if i<=6:
        path = folder + filename
        # extract faces
        face = extract_face(path)
        print(i, face.shape)
        # plot
        plt.subplot(3, 2, i)
        plt.axis('off')
        plt.imshow(face)
        i += 1
plt.show()

In [ ]:
path = './data_bw/val/Govinda/12.jpg'
face = extract_face(path)
print(1, face.shape)
# plot
plt.subplot(1, 3, 1)
plt.axis('off')
plt.imshow(face)
        
plt.show()

In [ ]:
# Each face is correctly detected
# Extract the faces for each subdirectory in the train and val folders, and prepare a dataset with names as output labels for each detected face.

# load faces
def load_faces(directory):
    faces  =  list()
    for filename in listdir(directory):
        # path
        path = directory + filename
        # extract face
        face = extract_face(path)
        # save result
        faces.append(face)
    return  faces

In [ ]:
# Take the input face as X and the label as y
def load_dataset(directory):
    X, y = list(), list()
    for subdir in listdir(directory):
        print(subdir)
        # path
        path = directory + subdir + '/'
        from os.path import isdir
        if not isdir(path):
            continue
        # load face
        faces = load_faces(path)
        # label
        labels = [subdir for _ in range(len(faces))]
        # save result
        X.extend(faces)
        y.extend(labels)
    return np.asarray(X), np.asarray(y)

In [ ]:
# train
trainX, trainy = load_dataset('./data_bw/train/')
print(trainX.shape, trainy.shape)

# show one picture
plt.imshow(trainX[0])

trainy[0]


In [ ]:
# vali
testX, testy = load_dataset('./data_bw/val/')
print(testX.shape, testy.shape)

# Save the result as a numpy npz format file
np.savez_compressed('./data_bw.npz', trainX, trainy, testX,  testy)